In [1]:
# !pip install PyGreSQL
# !pip install nbimporter

In [18]:
import os
import csv
import pgdb
import pickle
import zipfile
import nbimporter
import numpy as np
import pandas as pd
import scipy
# import ProviderPredictionScript

from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.multioutput import MultiOutputClassifier

In [3]:
class conn_args(object):
    def __init__(self, writehost, host, port, user, db, pw):
        self.writehost = writehost
        self.host = host
        self.port = port
        self.user = user
        self.db = db
        self.pp = pw

In [4]:
env = '/home/ec2-user/SageMaker/HM5_Common/env/standardized_connection_details.txt'
with open(env, 'r') as fIN:
    reader = csv.reader(fIN)
    for row in reader:
        if row[0].split('=')[0] == 'DB_DATABASE':
            db = row[0].split('=')[1]
        if row[0].split('=')[0] == 'DB_WRITEHOST':
            writehost = row[0].split('=')[1]
        if row[0].split('=')[0] == 'DB_HOST':
            host = row[0].split('=')[1]
        if row[0].split('=')[0] == 'DB_PORT':
            port = row[0].split('=')[1]
        if row[0].split('=')[0] == 'DB_PASSWORD':
            pw = row[0].split('=')[1]
        if row[0].split('=')[0] == 'DB_USERNAME':
            user = row[0].split('=')[1]

strConnArgs = conn_args(writehost, host, port, user, db, pw)

In [5]:
# conn = pgdb.connect(host=strConnArgs.host + ':' + strConnArgs.port,
#                     database=strConnArgs.db,
#                      user=strConnArgs.user, password=strConnArgs.pp)

# all_tables = "SELECT * FROM information_schema.tables;"
# all_tables_df = pd.read_sql(all_tables, conn)

# conn.close()

In [6]:
# conn = pgdb.connect(host=strConnArgs.host + ':' + strConnArgs.port,
#                     database=strConnArgs.db,
#                      user=strConnArgs.user, password=strConnArgs.pp)

# spec_query = "SELECT * FROM specialty_group;"
# spec_df = pd.read_sql(spec_query, conn)

# conn.close()

In [7]:
conn = pgdb.connect(host=strConnArgs.host + ':' + strConnArgs.port,
                    database=strConnArgs.db,
                     user=strConnArgs.user, password=strConnArgs.pp)

one_spec_query = "SELECT * FROM enginednlines_5;"
one_spec_df = pd.read_sql(one_spec_query, conn)

conn.close()

In [8]:
### How many providers do we want to synthesize?

## User defined for each specialty
num_providers = 1000

In [19]:
### Probability of number of claims per provider

### We'll eventually bring inference (of number of claims for each provider) from all customers for each specialty.

### Based on MonteCarlo simulation from the empirical data
num_claims_df = one_spec_df.groupby(['ren_id']).claim_id\
.nunique().reset_index().rename(columns = {'claim_id': 'num_claims'} )


num_iter = 10000
num_claims_avg_lst = []
for ii in range(0, num_iter):
    num_claims_avg_lst.append(round(num_claims_df.sample(round(0.1*len(num_claims_df))).num_claims.mean()))
dist_mean = np.round(np.mean(num_claims_avg_lst))
dist_std = np.round(np.std(num_claims_avg_lst))
group = [int(round(x)) for x in np.random.normal(loc=dist_mean, scale=3*dist_std,size=num_providers)]
scipy.stats.describe(group)

DescribeResult(nobs=1000, minmax=(11, 91), mean=53.308, variance=143.30243843843843, skewness=0.024267655007029112, kurtosis=-0.036427315296165386)

In [10]:
### Determine probability of different combinations 
### We'll eventually bring inference (of number of claims for each combination) from all customers for each specialty.
all_combinations = one_spec_df[['proc_values', 'place_of_service', 'pat_gender',
            'diag_values', 'mod_values', 'claim_id', 
            ]].fillna('None').groupby(['proc_values', 'place_of_service', 'pat_gender',
            'diag_values', 'mod_values'])\
.claim_id\
.nunique()\
.reset_index()\
.rename(columns = {'claim_id': 'num_claims'} )
all_combinations['prob_claims'] = all_combinations['num_claims']/len(all_combinations)
sel_combinations = all_combinations[all_combinations.prob_claims>=0.01]

In [24]:
## Unique combinations and corresponding weights
comb_list = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).values
weights = sel_combinations['prob_claims'].tolist()

In [26]:
import random  
final_df_list = []
random_outliers = random.sample(range(0, num_providers), int(round(0.01*num_providers)))
for ii in range(0, num_providers):
    if ii in random_outliers:
        tmp = pd.DataFrame(random.choices(comb_list, random.shuffle(weights), k=group[ii]), 
                       columns = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).columns)
    else:
        tmp = pd.DataFrame(random.choices(comb_list, weights, k=group[ii]), 
                       columns = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).columns)
    tmp['ren_id'] = 'prov_' + str(ii)
    tmp['prov_claim_id'] = ['claim_' + str(x) for x in range(0, group[ii])]
    final_df_list.append(tmp)
final_df = pd.concat(final_df_list)

In [27]:
final_df

,proc_values,place_of_service,pat_gender,diag_values,mod_values,ren_id,prov_claim_id
0,90834,30,None,F411,WD,prov_0,claim_0
1,90837,30,None,F341,WD,prov_0,claim_1
2,90837,30,None,F4322,WD,prov_0,claim_2
3,90837@90837,30,None,F4323,WD,prov_0,claim_3
4,90837,30,None,F411,WD,prov_0,claim_4
...,...,...,...,...,...,...,...
64,90837,30,None,F39,WD,prov_999,claim_64
65,90837@90837@90837,30,None,F419,WD,prov_999,claim_65
66,90837@90837@90837@90837@90837,30,None,F411,WD,prov_999,claim_66
67,90837,30,None,F909,WD,prov_999,claim_67
